# Chat Interface

> Interactive chat interface for natural language database queries

In [ ]:
#| default_exp chat_interface

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import sys
import sqlite3
from datetime import datetime
from typing import Dict, Any, Optional

# Check for required dependencies
try:
    from openai import OpenAI
    from llamaindex_mcp_nbdev.core import init_db
    from llamaindex_mcp_nbdev.llamaindex_integration import query_database_nl
    DEPENDENCIES_AVAILABLE = True
except ImportError as e:
    DEPENDENCIES_AVAILABLE = False
    print(f"Warning: {e}")
    print("Some features may not be available")

In [ ]:
#| export
class DatabaseChatbot:
    """Simple chatbot that can interact with SQLite database using natural language."""
    
    def __init__(self, db_path: str = "demo.db", api_key: Optional[str] = None):
        """Initialize the chatbot.
        
        Args:
            db_path: Path to SQLite database
            api_key: OpenAI API key (uses environment variable if not provided)
        """
        self.db_path = db_path
        self.client = None
        
        # Get API key
        self.api_key = api_key or os.environ.get("OPENAI_API_KEY")
        if not self.api_key:
            print("WARNING: No OpenAI API key found. Natural language queries will not work.")
            print("Set OPENAI_API_KEY environment variable or pass api_key parameter.")
        
        # Initialize OpenAI client if available
        if self.api_key and DEPENDENCIES_AVAILABLE:
            self.client = OpenAI(api_key=self.api_key)
        
        # Initialize database
        init_db(self.db_path)
        print(f"Connected to database: {self.db_path}")
        
    def show_sample_data(self) -> str:
        """Display current data in the database."""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("SELECT COUNT(*) FROM people")
        count = cursor.fetchone()[0]
        
        output = f"\nDatabase contains {count} records\n"
        
        if count > 0:
            output += "\nSample data (first 5 records):\n"
            cursor.execute("SELECT * FROM people LIMIT 5")
            results = cursor.fetchall()
            
            output += "ID | Name            | Age | Profession\n"
            output += "-" * 45 + "\n"
            for row in results:
                output += f"{row[0]:2d} | {row[1]:15s} | {row[2]:3d} | {row[3]}\n"
        
        conn.close()
        return output
    
    def add_person(self, name: str, age: int, profession: str) -> str:
        """Add a person to the database."""
        # Validate inputs
        if not name or not name.strip():
            return "Name cannot be empty"
        
        if not isinstance(age, int) or age < 1 or age > 150:
            return "Age must be between 1 and 150"
        
        if not profession or not profession.strip():
            return "Profession cannot be empty"
        
        # Add to database
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        try:
            cursor.execute(
                "INSERT INTO people (name, age, profession) VALUES (?, ?, ?)",
                (name.strip(), age, profession.strip())
            )
            conn.commit()
            conn.close()
            return f"Successfully added {name} to the database!"
        except Exception as e:
            conn.close()
            return f"Error adding person: {e}"
    
    def query_database(self, question: str) -> str:
        """Query database using natural language."""
        if not self.api_key:
            return "Error: OpenAI API key not configured"
        
        result = query_database_nl(question, self.db_path)
        
        if result.get("success"):
            response = f"\nAnswer: {result['answer']}"
            if result.get('sql_query'):
                response += f"\n\nGenerated SQL: {result['sql_query']}"
            return response
        else:
            return f"\nError: {result.get('error', 'Unknown error')}"

In [ ]:
#| export
def setup_sample_data(db_path: str = "demo.db") -> str:
    """Add sample data to database if empty."""
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Check if data exists
    cursor.execute("SELECT COUNT(*) FROM people")
    count = cursor.fetchone()[0]
    
    if count == 0:
        sample_data = [
            ("Alice Johnson", 28, "Developer"),
            ("Bob Smith", 35, "Engineer"),
            ("Carol Davis", 42, "Manager"),
            ("David Wilson", 23, "Developer"),
            ("Eve Brown", 31, "Designer"),
            ("Frank Miller", 45, "Engineer"),
            ("Grace Lee", 29, "Developer"),
            ("Henry Taylor", 38, "Manager")
        ]
        
        cursor.executemany(
            "INSERT INTO people (name, age, profession) VALUES (?, ?, ?)",
            sample_data
        )
        conn.commit()
        conn.close()
        return f"Added {len(sample_data)} sample records to database"
    else:
        conn.close()
        return f"Database already contains {count} records"

In [ ]:
#| export
def main():
    """Main entry point for chat interface."""
    # Setup sample data if needed
    setup_sample_data()
    
    chatbot = DatabaseChatbot()
    
    print("\n" + "="*50)
    print("SQLite Database Chat Assistant")
    print("="*50)
    print("Chat with your database using natural language!")
    print("Type 'exit' to quit")
    
    # Show initial data
    print(chatbot.show_sample_data())
    
    while True:
        try:
            user_input = input("\nYou: ").strip()
            
            if not user_input:
                continue
                
            if user_input.lower() in ['exit', 'quit', 'bye']:
                print("\nGoodbye!")
                break
                
            if user_input.lower() == 'show data':
                print(chatbot.show_sample_data())
                continue
                
            # Try natural language query
            response = chatbot.query_database(user_input)
            print(response)
            
        except KeyboardInterrupt:
            print("\n\nGoodbye!")
            break
        except Exception as e:
            print(f"\n Error: {e}")



In [ ]:
#| export
#| eval: false
if __name__ == "__main__":
    main()

In [ ]:
#| eval: false
# Setup sample data
print(setup_sample_data())

In [ ]:
#| eval: false
# Create chatbot instance
chatbot = DatabaseChatbot("demo.db")

# Show current data
print(chatbot.show_sample_data())

In [ ]:
#| eval: false
# Example: Add a new person
result = chatbot.add_person("John Doe", 30, "Data Scientist")
print(result)

In [ ]:
#| eval: false
# Example: Natural language query (requires OpenAI API key)
if os.environ.get("OPENAI_API_KEY"):
    response = chatbot.query_database("What's the average age of developers?")
    print(response)
else:
    print("Set OPENAI_API_KEY to test natural language queries")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()